In [2]:
import numpy as np
import pandas as pd

# data preparation

In [3]:
admissions = pd.read_csv('binary.csv')
print(admissions[0:10])

   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1
3      1  640  3.19     4
4      0  520  2.93     4
5      1  760  3.00     2
6      1  560  2.98     1
7      0  400  3.08     2
8      1  540  3.39     3
9      0  700  3.92     2


In [4]:
# make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
print(data[0:10])
data = data.drop('rank', axis=1)
print(data[0:10])

   admit  gre   gpa  rank  rank_1  rank_2  rank_3  rank_4
0      0  380  3.61     3       0       0       1       0
1      1  660  3.67     3       0       0       1       0
2      1  800  4.00     1       1       0       0       0
3      1  640  3.19     4       0       0       0       1
4      0  520  2.93     4       0       0       0       1
5      1  760  3.00     2       0       1       0       0
6      1  560  2.98     1       1       0       0       0
7      0  400  3.08     2       0       1       0       0
8      1  540  3.39     3       0       0       1       0
9      0  700  3.92     2       0       1       0       0
   admit  gre   gpa  rank_1  rank_2  rank_3  rank_4
0      0  380  3.61       0       0       1       0
1      1  660  3.67       0       0       1       0
2      1  800  4.00       1       0       0       0
3      1  640  3.19       0       0       0       1
4      0  520  2.93       0       0       0       1
5      1  760  3.00       0       1       0       

In [5]:
# standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:, field] = (data[field] - mean) / std
print(mean)
print(std)
print(data[0:10])

3.3899
0.38056677163
   admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
0      0 -1.798011  0.578348       0       0       1       0
1      1  0.625884  0.736008       0       0       1       0
2      1  1.837832  1.603135       1       0       0       0
3      1  0.452749 -0.525269       0       0       0       1
4      0 -0.586063 -1.208461       0       0       0       1
5      1  1.491561 -1.024525       0       1       0       0
6      1 -0.239793 -1.077078       1       0       0       0
7      0 -1.624876 -0.814312       0       1       0       0
8      1 -0.412928  0.000263       0       0       1       0
9      0  0.972155  1.392922       0       1       0       0


In [6]:
# split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
print(len(sample))
train_data, test_data = data.loc[sample], data.drop(sample)
# data, test_data = data.ix[sample], data.drop(sample)
print(test_data)

360
     admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
48       0 -1.278605 -2.390908       0       0       0       1
50       0  0.452749  1.235263       0       0       1       0
80       0  0.972155 -1.287291       0       0       0       1
84       1 -0.759199  0.552071       0       0       1       0
98       0  0.972155 -1.339844       0       1       0       0
110      0  0.799020 -0.814312       0       0       0       1
120      1 -0.586063  0.919944       0       1       0       0
122      0 -0.586063 -1.418674       0       0       1       0
133      0 -0.759199 -0.814312       0       0       1       0
148      1 -0.932334 -1.261014       1       0       0       0
169      0  0.106478  0.604625       0       0       1       0
184      0 -0.759199 -1.576333       0       0       0       1
188      0 -0.239793 -0.840588       0       1       0       0
202      1  0.972155  1.603135       1       0       0       0
204      1  0.106478  1.314093       1       0     

In [7]:
# split into features and targets
features_train, targets_train = train_data.drop('admit', axis=1), train_data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']
print(features_test)
print(targets_test)

          gre       gpa  rank_1  rank_2  rank_3  rank_4
48  -1.278605 -2.390908       0       0       0       1
50   0.452749  1.235263       0       0       1       0
80   0.972155 -1.287291       0       0       0       1
84  -0.759199  0.552071       0       0       1       0
98   0.972155 -1.339844       0       1       0       0
110  0.799020 -0.814312       0       0       0       1
120 -0.586063  0.919944       0       1       0       0
122 -0.586063 -1.418674       0       0       1       0
133 -0.759199 -0.814312       0       0       1       0
148 -0.932334 -1.261014       1       0       0       0
169  0.106478  0.604625       0       0       1       0
184 -0.759199 -1.576333       0       0       0       1
188 -0.239793 -0.840588       0       1       0       0
202  0.972155  1.603135       1       0       0       0
204  0.106478  1.314093       1       0       0       0
207  0.452749  0.630901       1       0       0       0
229  1.145291  0.079093       0       1       0 

# backpropagation

In [8]:
np.random.seed(21)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(h):
    return h * (1- h)

In [53]:
# hyperparameters
n_hidden = 2 # number of hidden units
epochs = 2000
learnrate = 0.1

In [40]:
n_records, n_features = features_train.shape
# print(features_train.shape)
last_loss = None

In [41]:
# initialize weights
weights_input_hidden = np.random.normal(loc=0.0, scale=1 / n_features ** 0.5, size=(n_features, n_hidden))
# loc(float):mean
# scale(float):standard deviation 
weights_hidden_output = np.random.normal(loc=0.0, scale=1 / n_features ** 0.5, size=n_hidden)
print(weights_input_hidden)
print(weights_hidden_output)

[[ 0.28247667 -0.18294243]
 [ 0.06627603  0.10501335]
 [-0.52070267  0.0261297 ]
 [-0.43350115 -0.40390796]
 [-0.18686472 -0.81003875]
 [-0.60275497  0.09463316]]
[ 0.26297692  0.34787763]


In [55]:
# backpropagation
for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features_train.values, targets_train):
        
        # forward pass
        hidden_input = np.dot(x, weights_input_hidden)
#         print(hidden_input)
        hidden_output = sigmoid(hidden_input)
#         print(hidden_output)
        output_input = np.dot(hidden_output, weights_hidden_output)
#         print(output_input)
        output = sigmoid(output_input)
#         print(output)
        
        # backward pass
        error = y - output
        
        del_output_error = error * sigmoid_prime(output_input)
        del_hidden_error = np.dot(del_output_error, weights_hidden_output) * sigmoid_prime(hidden_input)
        
        del_w_hidden_output += del_output_error * hidden_output
        del_w_input_hidden += del_hidden_error * x[:, None]
        
    weights_hidden_output += learnrate * del_w_hidden_output  / n_records
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    
    # print out the mean square error on the training set
    if e % 100 == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))
        loss = np.mean((output - y)**2)
        
        if last_loss and last_loss < loss:
            print(e, "train loss: ", loss, "WARNING loss increasiong")
        else:
            print(e, "train loss: ", loss)
        last_loss = loss

0 train loss:  0.1738112028 WARNING loss increasiong
100 train loss:  0.201420734607 WARNING loss increasiong
200 train loss:  0.223336387459 WARNING loss increasiong
300 train loss:  0.236146606128 WARNING loss increasiong
400 train loss:  0.242955042354 WARNING loss increasiong
500 train loss:  0.246450725319 WARNING loss increasiong
600 train loss:  0.248220629286 WARNING loss increasiong
700 train loss:  0.249111257126 WARNING loss increasiong
800 train loss:  0.249558139408 WARNING loss increasiong
900 train loss:  0.249782049559 WARNING loss increasiong


KeyboardInterrupt: 

# calculate accuracy on test data

In [52]:
print(weights_input_hidden)
print(weights_hidden_output)
hidden_output = sigmoid(np.dot(features_test, weights_input_hidden))
output = sigmoid(np.dot(hidden_output, weights_hidden_output))
predictions = output > 0.5 #???
accuracy = np.mean(predictions == targets_test)
print(format(accuracy))

[[ 0.1973686  -0.22571839]
 [-0.02179584  0.06661948]
 [-0.52844759  0.01903876]
 [-0.38680355 -0.39064005]
 [-0.16195884 -0.79609982]
 [-0.58592169  0.10637406]]
[-0.43545732 -0.42117413]
0.65
